In [ ]:
xgb_proba = xgb.predict_proba(X_test_full_scaled)[:, 1]

reconstructions = autoencoder.predict(X_test_full_scaled)
mse = np.mean(np.square(X_test_full_scaled- reconstructions), axis=1)

mse_norm = (mse - mse.min()) / (mse.max() - mse.min())

39767/39767 ━━━━━━━━━━━━━━━━━━━━ 83s 2ms/step


In [ ]:
hybrid_score = 0.3 * xgb_proba + 0.7 * mse_norm

In [ ]:
auc = roc_auc_score(y_test_full, hybrid_score)
print("Hybrid ROC–AUC:", auc)

# Threshold for classification
threshold = np.percentile(hybrid_score, 99)
y_pred_hybrid = (hybrid_score >= threshold).astype(int)

print("Hybrid Classification Report")
print(classification_report(y_test_full, y_pred_hybrid))


Hybrid ROC–AUC: 0.9999786682117888
Hybrid Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   1270881
           1       0.13      1.00      0.23      1643

    accuracy                           0.99   1272524
   macro avg       0.56      1.00      0.61   1272524
weighted avg       1.00      0.99      0.99   1272524



In [ ]:
stack_X = np.column_stack((xgb_proba, mse_norm))
stack_y = y_test_full

meta = LogisticRegression()
meta.fit(stack_X, stack_y)

hybrid_proba = meta.predict_proba(stack_X)[:, 1]

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# AE reconstruction error
reconstructions = autoencoder.predict(X_test_full_scaled, verbose=0)
mse = np.mean(np.square(X_test_full_scaled - reconstructions), axis=1)

# Define suspicious transactions based on AE threshold
threshold = np.percentile(mse, 90)
suspicious_mask = mse > threshold

# Initialize hybrid predictions
hybrid_preds = np.zeros(len(X_test_full_scaled), dtype=int)

# AE flags these as potentially fraudulent
hybrid_preds[suspicious_mask] = 1  # initially AE says "fraud"

# --- Step 4: Refine within the suspicious subset using XGB ---
X_suspect = X_test_full_scaled[suspicious_mask]
rf_proba = rf.predict_proba(X_suspect)[:, 1]

# XGB threshold for fraud (default 0.5, or tune via ROC)
rf_threshold = 0.5
rf_preds = (rf_proba >= rf_threshold).astype(int)

# Replace AE predictions for suspicious samples with XGB’s call
hybrid_preds[suspicious_mask] = rf_preds

# --- Step 5: Evaluation ---
print("Hybrid Confusion Matrix:")
print(confusion_matrix(y_test_full, hybrid_preds))

print("\nHybrid Classification Report:")
print(classification_report(y_test_full, hybrid_preds, digits=3))

roc_auc = roc_auc_score(y_test_full, hybrid_preds)
print(f"Hybrid ROC–AUC: {roc_auc:.6f}")


Hybrid Confusion Matrix:
[[1270867      14]
 [    393    1250]]

Hybrid Classification Report:
              precision    recall  f1-score   support

           0      1.000     1.000     1.000   1270881
           1      0.989     0.761     0.860      1643

    accuracy                          1.000   1272524
   macro avg      0.994     0.880     0.930   1272524
weighted avg      1.000     1.000     1.000   1272524

Hybrid ROC–AUC: 0.880396


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# SUPERVISED MODEL PREDICTIONS
xgb_proba = xgb.predict_proba(X_test_full_scaled)[:, 1]   # predicted fraud probabilities
xgb_preds = (xgb_proba >= 0.5).astype(int)

print("Supervised model results (XGB):")
print(classification_report(y_test_full, xgb_preds))
print("ROC–AUC (XGB):", roc_auc_score(y_test_full, xgb_proba))


# AUTOENCODER ANOMALY SCORING
reconstructions = autoencoder.predict(X_test_full_scaled)
mse = np.mean(np.square(X_test_full_scaled - reconstructions), axis=1)

# Normalize AE anomaly scores (0–1)
ae_score = (mse - mse.min()) / (mse.max() - mse.min())

# Binary anomaly flag (tune percentile)
threshold = np.percentile(mse, 99)
ae_flag = (mse >= threshold).astype(int)

print("\nAutoencoder (AE) Results:")
print(f"AE threshold (98th percentile): {threshold:.6f}")
print("Sample anomalies flagged:", np.sum(ae_flag))


# Rule-based hybrid: fraud if either model says so
hybrid_preds = np.where((xgb_preds + ae_flag) > 0, 1, 0)

# Weighted hybrid score
alpha = 0.3
hybrid_score = alpha * ae_score + (1 - alpha) * xgb_proba


print("\nHybrid (Rule-based) Results:")
print(classification_report(y_test_full, hybrid_preds))
print("ROC–AUC (Rule Hybrid):", roc_auc_score(y_test_full, hybrid_preds))

print("\nHybrid (Weighted Combination) Results:")
print(classification_report(y_test_full, (hybrid_score >= 0.5).astype(int)))
print("ROC–AUC (Weighted Hybrid):", roc_auc_score(y_test_full, hybrid_score))

cm = confusion_matrix(y_test_full, hybrid_preds)
print("\nHybrid Confusion Matrix:")
print(cm)



Supervised model results (XGB):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.69      1.00      0.82      1643

    accuracy                           1.00   1272524
   macro avg       0.85      1.00      0.91   1272524
weighted avg       1.00      1.00      1.00   1272524

ROC–AUC (XGB): 0.9999796921299604
39767/39767 ━━━━━━━━━━━━━━━━━━━━ 91s 2ms/step

Autoencoder (AE) Results:
AE threshold (98th percentile): 0.088669
Sample anomalies flagged: 12726

Hybrid (Rule-based OR logic) Results:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99   1270881
           1       0.11      1.00      0.20      1643

    accuracy                           0.99   1272524
   macro avg       0.56      0.99      0.60   1272524
weighted avg       1.00      0.99      0.99   1272524

ROC–AUC (Rule Hybrid): 0.9946397371283479

Hybrid (Weighted Combination) Results:
             

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# SUPERVISED MODEL PREDICTIONS
rf_proba = rf.predict_proba(X_test_full_scaled)[:, 1]   # predicted fraud probabilities
rf_preds = (rf_proba >= 0.5).astype(int)

print("Supervised model results (Random Forest):")
print(classification_report(y_test_full, rf_preds))
print("ROC–AUC (RF):", roc_auc_score(y_test_full, rf_proba))


# AUTOENCODER ANOMALY SCORING
reconstructions = autoencoder.predict(X_test_full_scaled)
mse = np.mean(np.square(X_test_full_scaled - reconstructions), axis=1)

# Normalize AE anomaly scores (0–1)
ae_score = (mse - mse.min()) / (mse.max() - mse.min())

# Binary anomaly flag (tune percentile)
threshold = np.percentile(mse, 99)
ae_flag = (mse >= threshold).astype(int)

print("\nAutoencoder (AE) Results:")
print(f"AE threshold (98th percentile): {threshold:.6f}")
print("Sample anomalies flagged:", np.sum(ae_flag))


# Rule-based hybrid: fraud if either model says so
hybrid_preds = np.where((rf_preds + ae_flag) > 0, 1, 0)

# Weighted hybrid score
alpha = 0.3
hybrid_score = alpha * ae_score + (1 - alpha) * xgb_proba


print("\nHybrid (Rule-based) Results:")
print(classification_report(y_test_full, hybrid_preds))
print("ROC–AUC (Rule Hybrid):", roc_auc_score(y_test_full, hybrid_preds))

print("\nHybrid (Weighted Combination) Results:")
print(classification_report(y_test_full, (hybrid_score >= 0.5).astype(int)))
print("ROC–AUC (Weighted Hybrid):", roc_auc_score(y_test_full, hybrid_score))

cm = confusion_matrix(y_test_full, hybrid_preds)
print("\nHybrid Confusion Matrix:")
print(cm)



Supervised model results (Random Forest):
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.93      1.00      0.96      1643

    accuracy                           1.00   1272524
   macro avg       0.96      1.00      0.98   1272524
weighted avg       1.00      1.00      1.00   1272524

ROC–AUC (RF): 0.9999995627515011
39767/39767 ━━━━━━━━━━━━━━━━━━━━ 48s 1ms/step

Autoencoder (AE) Results:
AE threshold (98th percentile): 0.088669
Sample anomalies flagged: 12726

Hybrid (Rule-based) Results:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00   1270881
           1       0.12      1.00      0.21      1643

    accuracy                           0.99   1272524
   macro avg       0.56      0.99      0.60   1272524
weighted avg       1.00      0.99      0.99   1272524

ROC–AUC (Rule Hybrid): 0.9948514013203533

Hybrid (Weighted Combination) Results:
             

In [ ]:
# AE-based anomaly score (normalized 0–1)
ae_score = (mse - mse.min()) / (mse.max() - mse.min())

# XGB fraud probability
xgb_proba = xgb.predict_proba(X_test_full_scaled)[:, 1]

# Weighted combination (tune alpha)
alpha = 0.3  # gives more weight to AE
hybrid_score = alpha * ae_score + (1 - alpha) * xgb_proba

# Threshold for fraud
hybrid_preds = (hybrid_score >= 0.5).astype(int)

print(classification_report(y_test_full, hybrid_preds))
print("Hybrid ROC–AUC:", roc_auc_score(y_test_full, hybrid_score))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1270881
           1       0.74      1.00      0.85      1643

    accuracy                           1.00   1272524
   macro avg       0.87      1.00      0.92   1272524
weighted avg       1.00      1.00      1.00   1272524

Hybrid ROC–AUC: 0.9999798372409079
